In [ ]:
# -*- coding: utf-8 -*-

# ipv6_calculator.py
# Autor: João Gabriel de Morais Bezerra
# Data: 18/06/2025

import random

def normalize_ipv6(address: str) -> str:
    """
    Expande um endereço IPv6 abreviado para sua forma completa de 32 dígitos hexadecimais.
    Exemplo: '::1' -> '0000:0000:0000:0000:0000:0000:0000:0001'

    Args:
        address (str): O endereço IPv6, possivelmente abreviado.

    Returns:
        str: O endereço IPv6 na sua forma completa.
    """
    # Se o endereço contém '::', ele está abreviado
    if '::' in address:
        # Divide o endereço em duas partes ao redor do '::'
        part1_str, part2_str = address.split('::')

        # Obtém os grupos de cada parte
        part1_groups = [group for group in part1_str.split(':') if group]
        part2_groups = [group for group in part2_str.split(':') if group]

        # Calcula quantos grupos de '0000' são necessários para preencher o meio
        num_zero_groups = 8 - (len(part1_groups) + len(part2_groups))

        # Junta as partes com os grupos de zero no meio
        full_groups = part1_groups + ['0000'] * num_zero_groups + part2_groups
    else:
        # Se não há '::', apenas divide pelos ':'
        full_groups = address.split(':')

    # Garante que cada grupo tenha 4 dígitos, preenchendo com zeros à esquerda (zfill)
    normalized_groups = [group.zfill(4) for group in full_groups]

    # Retorna o endereço completo com os grupos unidos por ':'
    return ':'.join(normalized_groups).lower()

def abbreviate_ipv6(address: str) -> str:
    """
    Aplica as regras de abreviação a um endereço IPv6.
    Regra 1: Remove zeros à esquerda em cada grupo.
    Regra 2: Substitui a maior sequência de grupos de zeros por '::' (apenas uma vez).

    Os conceitos de 'leftmost' e 'rightmost' se aplicam à estrutura do endereço:
    - O 'leftmost' (mais à esquerda) é o prefixo da rede.
    - O 'rightmost' (mais à direita) é o identificador da interface.
    A abreviação opera sobre a representação textual completa do endereço.

    Args:
        address (str): O endereço IPv6, preferencialmente na forma completa.

    Returns:
        str: O endereço IPv6 abreviado.
    """
    # Garante que estamos trabalhando com a forma completa para evitar erros
    full_address = normalize_ipv6(address)
    groups = full_address.split(':')

    # Encontra a maior sequência de grupos '0000' para substituir por '::'
    longest_seq_start = -1
    longest_seq_len = 0
    current_seq_start = -1
    current_seq_len = 0

    for i, group in enumerate(groups):
        if group == '0000':
            if current_seq_len == 0:
                current_seq_start = i
            current_seq_len += 1
        else:
            # Se a sequência atual for a maior até agora, salva sua posição e tamanho
            if current_seq_len > longest_seq_len:
                longest_seq_len = current_seq_len
                longest_seq_start = current_seq_start
            current_seq_len = 0 # Reseta o contador da sequência

    # Verifica a última sequência, caso o endereço termine com zeros
    if current_seq_len > longest_seq_len:
        longest_seq_len = current_seq_len
        longest_seq_start = current_seq_start

    # Se uma sequência de zeros com mais de um grupo foi encontrada, aplica a abreviação '::'
    if longest_seq_len > 1:
        start = longest_seq_start
        end = longest_seq_start + longest_seq_len
        # Constrói o endereço com '::'
        part1 = ':'.join([g.lstrip('0') or '0' for g in groups[:start]])
        part2 = ':'.join([g.lstrip('0') or '0' for g in groups[end:]])

        # Junta as partes, cuidando para não ter '::' no início ou fim desnecessariamente
        if not part1 and not part2: return '::'
        if not part1: return '::' + part2
        if not part2: return part1 + '::'
        return part1 + '::' + part2
    else:
        # Se não há sequência longa, apenas remove os zeros à esquerda de cada grupo
        abbreviated_groups = [g.lstrip('0') or '0' for g in groups]
        return ':'.join(abbreviated_groups)

def generate_random_ipv6(prefix_len: int) -> str:
    """
    Gera um endereço IPv6 Global Unicast Address (GUA) aleatório com um prefixo específico.
    De acordo com a RFC 4291, GUAs geralmente começam com o prefixo '2000::/3',
    o que significa que os 3 primeiros bits são '001'.

    Args:
        prefix_len (int): O tamanho do prefixo, deve ser 48 ou 54.

    Returns:
        str: Um endereço IPv6 aleatório no formato completo.
    """
    if prefix_len not in [48, 54]:
        raise ValueError("O prefixo deve ser /48 ou /54 para esta função.")

    # 1. Gerar o prefixo (parte 'leftmost')

    # Converte o endereço para um único inteiro de 128 bits para facilitar a manipulação
    address_int = 0

    # O primeiro hexteto (16 bits) para um GUA (prefixo 2000::/3) deve estar entre 2000 e 3FFF.
    # Isso corresponde a '001' nos 3 primeiros bits.
    first_hextet = random.randint(0x2000, 0x3FFF)
    address_int |= first_hextet << (128 - 16) # Desloca para a posição mais à esquerda

    # Gera o restante dos bits do prefixo de forma aleatória
    # Começamos do bit 16, pois o primeiro hexteto já foi gerado
    random_prefix_bits = prefix_len - 16
    if random_prefix_bits > 0:
        random_part = random.getrandbits(random_prefix_bits)
        address_int |= random_part << (128 - prefix_len)

    # 2. Gerar o identificador de interface (parte 'rightmost')
    host_bits_len = 128 - prefix_len
    host_part = random.getrandbits(host_bits_len)
    address_int |= host_part

    # Converte o inteiro de 128 bits de volta para o formato de endereço IPv6
    groups = []
    for i in range(8):
        # Extrai cada grupo de 16 bits
        shift = 112 - (i * 16)
        group = (address_int >> shift) & 0xFFFF
        groups.append(f"{group:04x}")

    return ':'.join(groups)

# --- Função Principal para Execução ---
def main():
    """
    Função principal que executa a calculadora e demonstra suas funcionalidades.
    """
    print("--- Calculadora IPv6 ---")

    # 4. Exemplo de Abreviação de Endereço (conforme solicitado)
    print("\n[1] Teste de Abreviação de Endereço:")
    original_address_with_prefix = "2801:0390:0080:0000:0100:0000:0000:ff00 /64"
    address_part, prefix_part = original_address_with_prefix.split(' ')

    print(f"Endereço Original com Prefixo: {original_address_with_prefix}")

    # Abrevia o endereço
    abbreviated_address = abbreviate_ipv6(address_part)

    print(f"Resultado Abreviado com Prefixo: {abbreviated_address} {prefix_part}")

    print("\n   Teste Adicional (zeros no meio):")
    another_address = "2001:0db8:0000:0000:0000:ff00:0042:8329"
    print(f"   Original: {another_address}")
    print(f"   Abreviado: {abbreviate_ipv6(another_address)}")

    # 4.1. Geração de Endereços Aleatórios (RFC-compliant)
    print("\n[2] Geração de Endereços IPv6 Aleatórios (Global Unicast):")

    # Gerar com prefixo /48
    random_ipv6_48 = generate_random_ipv6(48)
    abbreviated_random_48 = abbreviate_ipv6(random_ipv6_48)
    print(f"Endereço Aleatório com /48: {abbreviated_random_48}/48")

    # Gerar com prefixo /54
    random_ipv6_54 = generate_random_ipv6(54)
    abbreviated_random_54 = abbreviate_ipv6(random_ipv6_54)
    print(f"Endereço Aleatório com /54: {abbreviated_random_54}/54")


if __name__ == '__main__':
    main()

--- Calculadora IPv6 ---

[1] Teste de Abreviação de Endereço:
Endereço Original com Prefixo: 2801:0390:0080:0000:0100:0000:0000:ff00 /64
Resultado Abreviado com Prefixo: 2801:390:80:0:100::ff00 /64

   Teste Adicional (zeros no meio):
   Original: 2001:0db8:0000:0000:0000:ff00:0042:8329
   Abreviado: 2001:db8::ff00:42:8329

[2] Geração de Endereços IPv6 Aleatórios (Global Unicast):
Endereço Aleatório com /48: 3f94:709e:ea3a:c87f:6a6d:ccdd:95ed:2cf6/48
Endereço Aleatório com /54: 26aa:3f3d:d537:8650:eb31:a440:db63:4f3f/54
